# Email聚类程序
## Version 2.0
### 读取CSV文件，对Content聚类

In [3]:
import os

import hdbscan
import pandas as pd
from langchain import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,) 

from dotenv import load_dotenv

In [4]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
df=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
df.drop(df.columns[0],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  576 non-null    object
dtypes: object(3)
memory usage: 13.7+ KB


## 对问题进行聚类分类

In [ ]:
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 
# 将Content列转换为Document类型
docs=[Document(page_content=text) for text in df['Content']]
# 利用嵌入模型进行向量化，修改为你自己的嵌入模型调用方式
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=openai_api_key,
    azure_endpoint="https://pubwebpoc.openai.azure.com/",
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-07-01-preview",
    model = "text-embedding-ada-002"
    )
doc_result = embeddings.embed_documents(docs)
#embeddings = OpenAIEmbeddings(chunk_size=1000).embed_documents(docs)
#聚类分析
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(doc_result)
df['cluster']=hdb.labels_
# 删除不在群集中的文档
df = df.query("cluster != -1") 

## 利用chatgpt给生成的类别命名

In [ ]:
# 定义提示函数
def get_prompt():
    system_template = "你是一位摘要专家。你要帮我为描述写一个明确的标题。"
    human_template = '''使用以下描述，写一个能概括这些描述的标题。
    
    QUESSTION DESCRIPTION:{questions_str}
    
    QUESSTION TITLE:'''
    
    return ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template(human_template),
        ],
        input_variables=["questions"],
    )

#初始化大语言模型，修改为你自己的调用方式
llm=AzureChatOpenAI(
    openai_api_key=openai_api_key, 
    deployment_name="testchat",
    model_name="gpt-35-turbo",
    openai_api_version="2023-07-01-preview",
    temperature=0.7, 
    )
# 取出分类
for c in df.cluster.unique():s
    chain = LLMChain(llm=llm, prompt=get_prompt(), verbose=False)
    questions_str = "\n".join(
        [
            f"{questions['Content']}\n"
            for questions in df.query(f"cluster == {c}").to_dict(orient="records")
        ]
    )
    result = chain.run({"questions": questions_str,})
    df.loc[df.cluster == c, "questions_title"] = result

## 查看chatgpt为分类为1命名的标题

In [ ]:
c = 1
with pd.option_context("display.max_colwidth", None):
    print(df.query(f"cluster == {c}").questions_title.values[0])
    display(df.query(f"cluster == {c}").drop(columns=["questions_title"]).head())